# Start Date

In [1]:
# input
# period = 1
# start_date = '2023-07-01'

# 1. Import Packages and Data

In [2]:
from IPython.display import display
import re

In [3]:
import pandas as pd 
import numpy as np 
import infrastructure
import filepath 
import pygsheets
import datetime as dt

In [4]:
#manual input
df = pd.read_csv('SS_LC_merged_data.csv')

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,12,14,16,25,26,27,28,29,30,31,32,33,34,35,36,43,47,50,53,56,57) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
# SMS testing pipeline
sms_test = infrastructure.get_smartsheet('test_sms')
sms_test['Hitpath Offer ID'] = sms_test['Hitpath Offer ID'].astype(str)
# SMS offer sheet 
sms_offer = infrastructure.get_smartsheet('offers_sms')
sms_offer = sms_offer[sms_offer['Hitpath Offer ID'].isna() == False]
sms_offer['Hitpath Offer ID'] = sms_offer['Hitpath Offer ID'].astype(int).astype(str)
# Email offer sheet 
email_offer = infrastructure.get_smartsheet('offers_email')
# get mamba for shortcode 
mamba = infrastructure.get_mamba()
# Offer Testing Timeline 
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
# timeline = gc.open_by_url('https://docs.google.com/spreadsheets/d/16vrHMWs0ambcBJ1sC0SqpYVG8SOSWlbg1N22-bF49v8/edit#gid=423634805') 
# timeline_wks  = timeline[1].get_as_df()

/Users/nathan/Documents/SMS DMA Python/SMS DataSource/infrastructure.py:108: FutureWarning: The default value of regex will change from True to False in a future version.
  snakes['Hitpath Offer ID']=snakes['Hitpath Offer ID'].astype(str).str.replace('.0', '')


In [6]:
#la nina
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
lanina = gc.open_by_url('https://docs.google.com/spreadsheets/d/1obszkCQoE0ELOR1O0CrLVETUEmEIWlGuyAmK3FgWSJg/edit#gid=1099746391') 
lanina_wks  = lanina[1].get_as_df()

# 2. Segment Size

In [7]:
#input
offer_id_test = '13003'
test_offer_info = sms_test.loc[sms_test['Hitpath Offer ID'] == offer_id_test]
if(test_offer_info['Payout Type'].iloc[0] == 'RevShare'):
    payout = 1.00
else:
    payout = float(test_offer_info['$ Payout'].replace(r'[^0-9\.]','',regex=True))
if(payout  > 50.00):
    payout = payout/2
# payout = 8.00
test_offer_vertical = test_offer_info.loc[test_offer_info['Hitpath Offer ID'] == offer_id_test, 'Vertical'].iloc[0]
expected_revenue = payout*1.6+20

In [8]:
#last 60 days of data
last60days= pd.Timestamp(dt.date.today()-dt.timedelta(days=60)).strftime('%Y-%m-%d')
df = df[df['Date']>=last60days]
df['Hitpath_Offer_ID'] = df['Hitpath_Offer_ID'].astype(str)
df['Hitpath_Offer_ID'] = df['Hitpath_Offer_ID'].replace(r'\.0$', '', regex=True)
df = df[df['Send Strategy'] != 'AR']



In [9]:
df['Date'] = pd.to_datetime(df['Date'])

In [10]:
last_7_day = pd.Timestamp(df['Date'].max()-dt.timedelta(days=6)) 
last_7_day_df = df.loc[df['Date']>= last_7_day]
#combine
df1= last_7_day_df.groupby(['DP&Pub','Date','Segments'])['Delivered'].sum().reset_index()
avgDelivered= df1.groupby(['Segments'])['Delivered'].median().reset_index()

In [11]:
# #median delivered volume by shortcode_DP.SV
# median_delivered = df.groupby(['Date','shortcode_DP.SV'])['Delivered'].sum().reset_index()
# median_delivered = median_delivered.groupby(['shortcode_DP.SV'])['Delivered'].median().reset_index()

In [12]:
#eCPM by shortcode_DP.SV
eCPM_df = df.groupby('shortcode_DP.SV').agg({'Revenue':'sum','Delivered':'sum'})
eCPM_df['eCPM'] = eCPM_df['Revenue'] * 1000 / eCPM_df['Delivered']
eCPM_df = eCPM_df.reset_index()

In [13]:
eCPM_df = eCPM_df.rename(columns={'Delivered': 'Total Delivered'})
avgDelivered = avgDelivered.rename(columns={'Delivered': 'Median Delivered'})

In [14]:
def extract_shortcode(segment):
    # Split the segment by underscore and take up to the first two parts
    parts = segment.split('_')
    # Rejoin the first two parts with an underscore if there are at least two parts, else return the first part
    return '_'.join(parts[:2])

# Apply this function to each segment in avgDelivered
avgDelivered['shortcode_DP.SV'] = avgDelivered['Segments'].apply(extract_shortcode)

In [15]:
terms_to_exclude = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', '(?<!N)EXL', 'NT']
pattern = '|'.join([f'({term})' for term in terms_to_exclude])
avgDelivered_filtered = avgDelivered[~avgDelivered['Segments'].str.contains(pattern, regex=True, case=False)]

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [16]:
df_filtered = df[df['Delivered'] > 0]
df_filtered['Date'] = pd.to_datetime(df_filtered['Date'])
avgDelivered_date_merged = pd.merge(avgDelivered_filtered, df_filtered, on='Segments', how='left')
avgDelivered_date_merged.rename(columns={'shortcode_DP.SV_x': 'shortcode_DP.SV'}, inplace=True)
avgDelivered_date_merged = avgDelivered_date_merged.sort_values(by=['shortcode_DP.SV', 'Date'], ascending=[True, False])
avgDelivered_latest = avgDelivered_date_merged.drop_duplicates(subset=['shortcode_DP.SV'], keep='first')

# Step 4: Merge the prepared segment information with eCPM_df
merged_df = pd.merge(eCPM_df, avgDelivered_latest, on='shortcode_DP.SV', how='left')

<ipython-input-16-3107b4e9a219>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Date'] = pd.to_datetime(df_filtered['Date'])


In [17]:
merged_df = merged_df[['shortcode_DP.SV','Segments','Median Delivered','Revenue_x','Total Delivered',
                       'eCPM']]
merged_df = merged_df.rename(columns= {"Revenue_x": "Revenue"})

In [18]:
merged_df['segment_size'] = expected_revenue * 1000 / merged_df['eCPM']
merged_df['segment_size'] -= merged_df['segment_size'] % -100

In [19]:
merged_df

,shortcode_DP.SV,Segments,Median Delivered,Revenue,Total Delivered,eCPM,segment_size
0,DSS_JET.ZTA,DSS_JET.ZTA_1PLD.30DC,6530.5,4484.9500,471730.0,9.507451,3400.0
1,DSS_RHD.CC,DSS_RHD.CC_30DC_TMO,2270.5,883.8900,110043.0,8.032224,4000.0
2,DSS_TLG.PL,DSS_TLG.PL_30DC,1287.0,262.8700,8779.0,29.943046,1100.0
3,DSS_ZM.PL.2,DSS_ZM.PL.2_30DC,4296.5,2995.1830,251151.0,11.925825,2700.0
4,FLC_AI.CC,NaN,NaN,122.3500,4933.0,24.802352,1300.0
5,FLC_EDM.247L,FLC_EDM.247L_30DC_NEXL,6634.0,14752.2850,591446.0,24.942742,1300.0
6,FLC_I.CC,NaN,NaN,165.8500,4923.0,33.688808,1000.0
7,FLC_I.RC,FLC_I.RC_30DC_VZN,7636.5,6615.9860,381918.0,17.323054,1900.0
8,FLC_JET.ZTA,FLC_JET.ZTA_30DC_VZN,167.5,276.1800,15046.0,18.355709,1800.0
9,FLC_NPD.RTO,FLC_NPD.RTO_7DC_VZN,225.0,306.9645,9184.0,33.423835,1000.0


# 3. Offer Vertical

In [20]:
#merge vertical with offer per drop
sms_offer = sms_offer.rename(columns={"Hitpath Offer ID": "Hitpath_Offer_ID"})
vertical_merge = df.merge(sms_offer[['Hitpath_Offer_ID','Vertical']],on='Hitpath_Offer_ID', how = 'left')

In [21]:
#vertical eCPM by shortcode_dp.sv
vertical_eCPM_DF = vertical_merge.groupby(['shortcode_DP.SV','Vertical']).agg(
    {'Revenue':'sum','Delivered':'sum'}).reset_index()
vertical_eCPM_DF['Offer Vertical eCPM'] = vertical_eCPM_DF['Revenue'] * 1000 / vertical_eCPM_DF['Delivered']

# 4. Testing

In [22]:
# #manual vertical if not enough data/payout
# test_offer_vertical = 'Education'

In [23]:
#similar buckets
bucket1 = ['Loan', 'Debt', 'Credit Score', 
           'Credit Card', 'Financial', 'Shopping', 'Business Opportunity', 'Bank Account', 'Tax Debt']
bucket2 = ['Cellular', 'App Install', 'Sweepstakes', 'Gift Card', 'Gas Card', 'Freebies', 'Surveys', 'Promos']
bucket3 = ['Education', 'Benefits','Resources', 'Rent To Own', 'Employment','Delivery', 'Miscellaneous', 
           'Tort', 'Legal', 'Injury Claim', 'Identity Protection']
bucket4 = ['Auto Warranty', 'Auto Loan', 'Auto Insurance', 'Auto Purchase', 
           'Home Security', 'Health', 'Healthcare', 'Home Loan', 'Home Warranty', 'Home Refinance']



In [24]:
buckets = pd.DataFrame({'bucket1': pd.Series(bucket1), 'bucket2': pd.Series(bucket2), 'bucket3': pd.Series(bucket3),
             'bucket4': pd.Series(bucket4)})

In [25]:
vertical_eCPM = vertical_merge.groupby('Vertical').agg({'Revenue':'sum','Delivered':'sum'}).reset_index()
vertical_eCPM['Vertical eCPM'] = vertical_eCPM['Revenue'] * 1000 / vertical_eCPM['Delivered']
vertical_eCPM

,Vertical,Revenue,Delivered,Vertical eCPM
0,Auto Insurance,167.94,21421.0,7.839970
1,Auto Warranty,5713.00,389472.0,14.668577
2,Cellular,252.00,41323.0,6.098299
3,Credit Card,20162.85,1583555.0,12.732649
4,Credit Score,351.20,27188.0,12.917464
5,Education,9204.00,805495.0,11.426514
6,Employment,4260.00,355033.0,11.998885
7,Health,0.00,5950.0,0.000000
8,Healthcare,40.00,19287.0,2.073936
9,Home Financing,151.10,32910.0,4.591310


In [32]:
if 'test' in locals():
    del test
if 'test2' in locals():
    del test2
if((vertical_eCPM.loc[vertical_eCPM['Vertical'] == test_offer_vertical, 'Delivered'].size == 1) == True):
    if(vertical_eCPM.loc[vertical_eCPM['Vertical'] == test_offer_vertical, 'Delivered'].iloc[0]) < 100000:
        print("Running Block 1")
        for column in buckets:
            if(buckets[column].eq(test_offer_vertical)).any():
                sim = buckets[column]
                bucket = vertical_eCPM_DF.loc[vertical_eCPM_DF['Vertical'].isin(sim)]
                bucket_merged = bucket.merge(merged_df[['shortcode_DP.SV','segment_size']],on='shortcode_DP.SV', how = 'left')
                bucket_merged = bucket_merged.merge(avgDelivered_filtered[['shortcode_DP.SV','Median Delivered']],on='shortcode_DP.SV', how = 'left')
                bucket_merged = bucket_merged.rename(columns={"Delivered": "Total Delivered", "Delivered_y": "Median Delivered"})
                bucket_merged['% Testing Size'] = bucket_merged['segment_size']/bucket_merged['Median Delivered']
                test = bucket_merged.loc[((bucket_merged['Median Delivered'] < 10000) & 
                          (bucket_merged['Median Delivered'] >= 4000) & 
                          (bucket_merged['% Testing Size'] <= .35)) |
                          ((bucket_merged['Median Delivered'] >= 10000) &
                          (bucket_merged['% Testing Size'] <= .2))
                         ].sort_values(by = "Offer Vertical eCPM",ascending = False)
                test2 = test.groupby('shortcode_DP.SV').agg({'Revenue':'sum','Total Delivered':'sum'}).reset_index()
                test2['Bucket eCPM'] = test2['Revenue'] * 1000 / test2['Total Delivered']
                test2 = test2.merge(merged_df[['shortcode_DP.SV','segment_size']],on='shortcode_DP.SV', how = 'left')
                test2 = test2.merge(avgDelivered_filtered[['shortcode_DP.SV','Median Delivered']],on='shortcode_DP.SV', how = 'left')
                test2['% Testing Size'] = test2['segment_size']/test2['Median Delivered']
                test2 = pd.merge(test2, avgDelivered_filtered[['shortcode_DP.SV', 'Segments']], on='shortcode_DP.SV', how='left')


                
            else:
                print("Bucket not found in " + column)
    else:
        print("Running Block 2")
        bucket = vertical_eCPM_DF.loc[vertical_eCPM_DF['Vertical'] == test_offer_vertical]
        bucket_merged = bucket.merge(merged_df[['shortcode_DP.SV','segment_size']],on='shortcode_DP.SV', how = 'left')
        bucket_merged = bucket_merged.merge(avgDelivered_filtered[['shortcode_DP.SV','Median Delivered']],on='shortcode_DP.SV', how = 'left')
        bucket_merged = bucket_merged.rename(columns={"Delivered": "Total Delivered", "Delivered_y": "Median Delivered"})
        bucket_merged['% Testing Size'] = bucket_merged['segment_size']/bucket_merged['Median Delivered']
        test = bucket_merged.loc[((bucket_merged['Median Delivered'] < 10000) & 
                      (bucket_merged['Median Delivered'] >= 4000) & 
                      (bucket_merged['% Testing Size'] <= .35)) |
                      ((bucket_merged['Median Delivered'] >= 10000) &
                      (bucket_merged['% Testing Size'] <= .2))
                     ].sort_values(by = "Offer Vertical eCPM",ascending = False)
        test = pd.merge(test, avgDelivered_filtered[['shortcode_DP.SV', 'Segments']], on='shortcode_DP.SV', how='left')

else: 
    print("Running Block 3")
    for column in buckets:
            if(buckets[column].eq(test_offer_vertical)).any():
                sim = buckets[column]
                bucket = vertical_eCPM_DF.loc[vertical_eCPM_DF['Vertical'].isin(sim)]
                bucket_merged = bucket.merge(merged_df[['shortcode_DP.SV','segment_size']],on='shortcode_DP.SV', how = 'left')
                bucket_merged = bucket_merged.merge(avgDelivered[['shortcode_DP.SV','Median Delivered']],on='shortcode_DP.SV', how = 'left')
                bucket_merged = bucket_merged.rename(columns={"Delivered": "Total Delivered", "Delivered_y": "Median Delivered"})
                bucket_merged['% Testing Size'] = bucket_merged['segment_size']/bucket_merged['Median Delivered']
                test = bucket_merged.loc[((bucket_merged['Median Delivered'] < 10000) & 
                          (bucket_merged['Median Delivered'] >= 4000) & 
                          (bucket_merged['% Testing Size'] <= .35)) |
                          ((bucket_merged['Median Delivered'] >= 10000) &
                          (bucket_merged['% Testing Size'] <= .2))
                         ].sort_values(by = "Offer Vertical eCPM",ascending = False)
                test2 = test.groupby('shortcode_DP.SV').agg({'Revenue':'sum','Total Delivered':'sum'}).reset_index()
                test2['Bucket eCPM'] = test2['Revenue'] * 1000 / test2['Total Delivered']
                test2 = test2.merge(merged_df[['shortcode_DP.SV','segment_size']],on='shortcode_DP.SV', how = 'left')
                test2 = test2.merge(avgDelivered_filtered[['shortcode_DP.SV','Median Delivered']],on='shortcode_DP.SV', how = 'left')
                test2['% Testing Size'] = test2['segment_size']/test2['Median Delivered']
                test2 = pd.merge(test2, avgDelivered_filtered[['shortcode_DP.SV', 'Segments']], on='shortcode_DP.SV', how='left')

            else:
                print("Bucket not found in " + column)
        

Running Block 1
Bucket not found in bucket1
Bucket not found in bucket2
Bucket not found in bucket3


In [33]:
try:
    # Try to sort and display test2
    display(test2.sort_values(by="Bucket eCPM", ascending = False))
except NameError:
    # If test2 does not exist, sort and display test instead
    display(test.sort_values(by="Bucket eCPM", ascending = False))


,shortcode_DP.SV,Revenue,Total Delivered,Bucket eCPM,segment_size,Median Delivered,% Testing Size,Segments
6,MBC_PA.PS,1066.01,51058.0,20.878413,1500.0,4879.5,0.307409,MBC_PA.PS_30DC
0,FLC_EDM.247L,782.40,43744.0,17.885881,1300.0,6634.0,0.195960,FLC_EDM.247L_30DC_NEXL
3,FLC_PN.SWP,4100.00,231921.0,17.678434,2100.0,36365.5,0.057747,FLC_PN.SWP_37DC
10,UAA_SPK.CR2,1536.00,93999.0,16.340599,2700.0,13931.0,0.193812,UAA_SPK.CR2_30DC
7,MBC_PN.FC,1445.33,102903.0,14.045557,1500.0,13176.0,0.113843,MBC_PN.FC_21DC
5,MBC_I.RC,1360.00,100989.0,13.466813,2400.0,15459.5,0.155244,MBC_I.RC_15DC
1,FLC_I.RC,280.00,22118.0,12.659372,1900.0,7636.5,0.248805,FLC_I.RC_30DC_VZN
9,SVT_AL.PL.2,224.00,26807.0,8.356026,2600.0,7620.5,0.341185,SVT_AL.PL.2_30DC
8,MBC_PN.SWP,2057.00,262869.0,7.825190,3100.0,20976.5,0.147784,MBC_PN.SWP_1PLD.30DC_TMO
4,MBC_EDM.247L,175.00,26737.0,6.545237,2400.0,7259.5,0.330601,MBC_EDM.247L_21DC_TMO_NEXL


In [31]:
test2.to_clipboard()